## install software and data

In [ ]:
!pip -q install neurom --upgrade
!pip uninstall -y nsg-bluepyopt-job
!pip -q install https://github.com/jdcourcol/InSilicoNeuroscienceCourse/raw/master/nsg-portal-course/utils/nsg-bluepyopt-job-0.0.1.dev0.tar.gz
!mkdir -p ./tmp/
!curl https://github.com/jdcourcol/InSilicoNeuroscienceCourse/raw/master/nsg-portal-course/model/initial_data.tar.gz| tar x -C ./tmp
WORKING_DIR = './tmp/hippo-ca1'
    

In [ ]:
%load_ext autoreload
%autoreload
import getpass
import os.path
from __future__ import print_function
import pprint
pp = pprint.PrettyPrinter(indent=2)

Enable the code below to enable debug level logging

In [ ]:
# import logging                                                                      
# logging.basicConfig()                                                               
# logger = logging.getLogger()                                                        
# logger.setLevel(logging.DEBUG)   

## Model description

### Morphology

We're using a complex reconstructed morphology of an L5PC cell. Let's visualise this with the BlueBrain NeuroM software:

In [ ]:
import neurom.viewer
neurom.viewer.draw(neurom.load_neuron(os.path.join(WORKING_DIR, 'morphology', '010710HP2.asc')));

### Parameters

In [ ]:
import json
param_configs = json.load(open(os.path.join(WORKING_DIR,'config.orig/parameters.json')))
print(param_configs)

## Protocols

Now that we have a cell model, we can apply protocols to it. The protocols are also stored in a json file.

In [ ]:
proto_configs = json.load(open(os.path.join(WORKING_DIR, 'config.orig/protocols.json')))
print(proto_configs)

## eFeatures

For every protocol we need to define which eFeatures will be used as objectives of the optimisation algorithm.

In [ ]:
feature_configs = json.load(open(os.path.join(WORKING_DIR,'config.orig/features.json')))
pp.pprint(feature_configs)

modify configuration here...

## save configuration

In [ ]:
for variable, filename in zip([param_configs, proto_configs, feature_configs],['parameters.json', 'protocols.json', 'features.json']):
    with open(os.path.join(WORKING_DIR, 'config', filename), 'w') as f:
        json.dump(variable, f)
!tar -cf './bluepyopt-config.tar.gz' './tmp/hippo-ca1'

## launch optimization on the nsg-portal compute center

In [ ]:
user = #'put your nsg_portal account id here'

In [ ]:
password = getpass.getpass('provide your password:')

In [ ]:
import nsg_bluepyopt_job.job as job

In [ ]:
job_url = job.launch_opt(user, password, './bluepyopt-config.tar.gz')

In [ ]:
output_url = job.wait_completion(job_url, user, password, 30)

In [ ]:
checkpoint_pkl = job.download_output(output_url, './tmp/')